In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kesava2001","key":"1a54208c031894e754f3101adbc99f5a"}'}

In [3]:
!mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
!kaggle datasets download -d aryashah2k/breast-ultrasound-images-dataset

 89% 173M/195M [00:01<00:00, 80.4MB/s]
100% 195M/195M [00:01<00:00, 107MB/s] 


In [6]:
! unzip breast-ultrasound-images-dataset.zip

Archive:  breast-ultrasound-images-dataset.zip
  inflating: Dataset_BUSI_with_GT/benign/benign (1).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (1)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (10).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (10)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (100).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (100)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (100)_mask_1.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (101).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (101)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (102).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (102)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (103).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (103)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (104).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (104)_mask.png  
  inflating: Data

In [7]:
import os
data_path = '/content/Dataset_BUSI_with_GT'

categories = os.listdir(data_path)

labels = [i for i in range(len(categories))]

label_dict = dict(zip(categories, labels))

In [8]:
categories

['normal', 'malignant', 'benign']

In [9]:
import cv2
from google.colab.patches import cv2_imshow
target_size = 224
data = []
label = []

for category in categories:
  folder_path = os.path.join(data_path, category)
  image_titles = os.listdir(folder_path)
  for name in image_titles:
    image_path = os.path.join(folder_path, name)
    image = cv2.imread(image_path)
    try:
      resize = cv2.resize(image, (target_size, target_size))

      data.append(resize)

      label.append(label_dict[category])
    except Exception as e:
      print('exception', e)

In [10]:
from tensorflow import keras
from keras.utils import np_utils
encoded_label = np_utils.to_categorical(label)
encoded_label

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [12]:
import numpy as np

data = np.array(data)
Data = np.reshape(data, (data.shape[0], target_size, target_size, 3))
encoded_label = np.array(encoded_label)

In [13]:
Data=Data/255

In [14]:
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense

vgg = VGG16(input_shape=[224, 224, 3], weights='imagenet', include_top=False)

for layer in vgg.layers:
  layer.trainable = False

x = Flatten()(vgg.output)

prediction = Dense(3, 'softmax')(x)

58900480/58889256 [==============================] - 0s 0us/step


In [15]:
from keras.models import Model
model = Model(inputs=vgg.input, outputs=prediction)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [21]:
model.fit(Data, encoded_label, epochs=2, validation_split=0.2, verbose=1, batch_size=16)

Epoch 1/2
79/79 [==============================] - 7s 85ms/step - loss: 0.0703 - accuracy: 0.9818 - val_loss: 0.2308 - val_accuracy: 0.9082
Epoch 2/2
79/79 [==============================] - 7s 84ms/step - loss: 0.0466 - accuracy: 0.9897 - val_loss: 0.3130 - val_accuracy: 0.8892


In [38]:
import cv2
from google.colab.patches import cv2_imshow

img = cv2.imread('/content/Dataset_BUSI_with_GT/benign/benign (10).png')
img = cv2.resize(img, (224, 224))
img = np.reshape(img, (1, 224, 224, 3))

In [33]:
categories

['normal', 'malignant', 'benign']

In [39]:
categories[np.argmax(model.predict(img))]

'benign'

In [40]:
model.save('/content/drive/MyDrive/Colab Notebooks/AI-MAJOR-MAY/trained_breast_cancer.h5')